# Federated learning: random seed
This notebook is a copy of the notebook [Federated learning basic concepts](./federated_learning_basic_concepts.ipynb). The difference is that, here, we set a seed using [Reproducibility](https://github.com/sherpaai/Sherpa.ai-Federated-Learning-Framework/blob/master/shfl/private/reproducibility.py) Singleton Class, in order to ensure the reproducibility of the experiment. If you execute this experiment many times, you should always obtain the same results. 
However, apart from that, the structure is identical so the text has been removed for clearness. Please refer to the original notebook for the detailed description of the experiment. 

In [ ]:
from shfl.private.reproducibility import Reproducibility

# Server
Reproducibility(1234)

# In case of client
# Reproducibility.get_instance().set_seed(ID)

## The data

In [ ]:
import matplotlib.pyplot as plt
import shfl

database = shfl.data_base.Emnist()
train_data, train_labels, test_data, test_labels = database.load_data()

print(len(train_data))
print(len(test_data))
print(type(train_data[0]))
train_data[0].shape

plt.imshow(train_data[0])

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_labels = iid_distribution.get_nodes_federation(num_nodes=20, percent=10)

print(type(federated_data))
print(federated_data.num_nodes())
federated_data[0].private_data

## The model

In [ ]:
import tensorflow as tf

def model_builder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    loss = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.RMSprop()
    metrics = [tf.keras.metrics.categorical_accuracy]
    
    return shfl.model.DeepLearningModel(model=model, loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
federated_government = shfl.federated_government.FederatedGovernment(model_builder(), federated_data, aggregator)

In [ ]:
import numpy as np


def reshape_data(labeled_data):
    labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))

def normalize_data(data, mean, std):
    data.data = (data.data - mean) / std
        
federated_data.apply_data_transformation(reshape_data);

mean = np.mean(train_data.data)
std = np.std(train_data.data)
federated_data.apply_data_transformation(normalize_data, mean=mean, std=std);

## Run the federated learning experiment

In [ ]:
test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))
federated_government.run_rounds(3, test_data, test_labels)